# Introduction 

This note evaluate smolLM-135M instruct model with llama.cpp

# Env and tools prepare

In [1]:
!cp -r /kaggle/input/llama-cpp-binary-compiled-with-gpu/llama.cpp /kaggle/working/

In [2]:
!chmod +x -R llama.cpp
%cd llama.cpp
!cp -r /usr/local/cuda-12.3/targets /usr/local/nvidia/ 

/kaggle/working/llama.cpp


# Run perplexity

Perplexity is the main evaluation metric for large language models (LLMs). It measures how well the model predicts a given sequence of tokens.
Formally, the perplexity of LLMs is the exponentiated average negative log-likelihood,  which means **perplexity smaller is better** 

Perplexity benchmark shows how accurate the model is 

In [3]:

!/kaggle/working/llama.cpp/llama-perplexity -m  /kaggle/input/smollm-135m-instruct-gguf/gguf/q4_k_m/1/SmolLM-135M-Instruct-Q4_K_M-v2.gguf -f /kaggle/working/llama.cpp/wikitext-2-raw/wiki.test.raw --chunks 256 > ppl_Q4_V2.log 2>&1
!/kaggle/working/llama.cpp/llama-perplexity -m  /kaggle/input/smollm-135m-instruct-gguf/gguf/q4_k_m/1/SmolLM-135M-Instruct-Q4_K_M.gguf -f /kaggle/working/llama.cpp/wikitext-2-raw/wiki.test.raw --chunks 256 > ppl_Q4.log 2>&1
!/kaggle/working/llama.cpp/llama-perplexity -m  /kaggle/input/smollm-135m-instruct-gguf/gguf/fp16/1/SmolLM-135M-Instruc-fp16.gguf -f /kaggle/working/llama.cpp/wikitext-2-raw/wiki.test.raw --chunks 256 > pip_fp16.log 2>&1


In [4]:
!tail -6 ppl_Q4_V2.log
!tail -6 ppl_Q4.log
!tail -6 pip_fp16.log

Final estimate: PPL = 23.2651 +/- 0.28000

llama_perf_context_print:        load time =    1361.88 ms
llama_perf_context_print: prompt eval time =   29115.58 ms / 131072 tokens (    0.22 ms per token,  4501.78 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   49144.05 ms / 131073 tokens
Final estimate: PPL = 23.2651 +/- 0.28000

llama_perf_context_print:        load time =    1328.80 ms
llama_perf_context_print: prompt eval time =   28861.40 ms / 131072 tokens (    0.22 ms per token,  4541.43 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   48036.22 ms / 131073 tokens
Final estimate: PPL = 21.2879 +/- 0.25264

llama_perf_context_print:        load time =    2577.67 ms
llama_perf_context_print: prompt eval 

# Benchmarking the Inference Throughput and Memory Consumption 

Llama-bench shows performance and resource consumption. 
There are three types of test
- Prompt processing (pp): processing a prompt in batches (-p)
- Text generation (tg): generating a sequence of tokens (-n)
- Prompt processing + text generation (pg): processing a prompt followed by generating a sequence of tokens (-pg)

In the following test, we run text generation and prompt processing


In [5]:
!/kaggle/working/llama.cpp/llama-bench -m /kaggle/input/smollm-135m-instruct-gguf/gguf/q4_k_m/1/SmolLM-135M-Instruct-Q4_K_M-v2.gguf -m /kaggle/input/smollm-135m-instruct-gguf/gguf/q4_k_m/1/SmolLM-135M-Instruct-Q4_K_M.gguf -m /kaggle/input/smollm-135m-instruct-gguf/gguf/fp16/1/SmolLM-135M-Instruc-fp16.gguf -mg 1 -ngl 10,20,30,35 -n 128,256,512 

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    no
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: Tesla P100-PCIE-16GB, compute capability 6.0, VMM: yes
| model                          |       size |     params | backend    | ngl |   main_gpu |          test |                  t/s |
| ------------------------------ | ---------: | ---------: | ---------- | --: | ---------: | ------------: | -------------------: |
| llama ?B Q4_K - Medium         | 127.55 MiB |   162.83 M | CUDA       |  10 |          1 |         pp512 |      7222.64 ± 85.29 |
| llama ?B Q4_K - Medium         | 127.55 MiB |   162.83 M | CUDA       |  10 |          1 |         tg128 |         86.45 ± 4.96 |
| llama ?B Q4_K - Medium         | 127.55 MiB |   162.83 M | CUDA       |  10 |          1 |         tg256 |         88.31 ± 0.79 |
| llama ?B Q4_K - Medium         | 127.55 MiB |   162.83 M | CUDA       |  10 |          1 |         tg512 |         82.95 ± 2.48 |
| llama ?B Q4_K -